In [3]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd

## Atividade 1

>**Implemente uma classe que corresponda ao KNN**

In [4]:

class KNN:

    def __init__(self, k=3):

        self.k = k

    def fit(self, X_train, y_train):

        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test, distance_metric='l2'):

        if distance_metric == 'l1':

            dist_func = self._l1_distance

        elif distance_metric == 'l2':

            dist_func = self._l2_distance

        else:
            raise ValueError(
                "Métrica de distancia inválida. Escolha 'l1' ou 'l2'.")

        predictions = [self._predict(x, dist_func) for x in X_test]

        return np.array(predictions)

    def _predict(self, x, dist_func):

        distances = [dist_func(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def _l1_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))

    def _l2_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

## Atividade 2

**Agora implemente a classe que corresponde a uma Árvore de decisão**

In [5]:
import numpy as np


class DecisionTree:

    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2):

        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def fit(self, X, y):

        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):

        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))

        # Condições de parada
        if depth == self.max_depth or num_samples < self.min_samples_split or num_classes == 1:
            return np.bincount(y).argmax()

        # Escolha de critério de impureza
        if self.criterion == 'gini':
            impurity_func = self._gini_impurity
        elif self.criterion == 'entropy':
            impurity_func = self._entropy_impurity
        else:
            raise ValueError("Critério inválido. Escolha 'gini' ou 'entropy'.")

        #  construa a árvore de decisão recursivamente, dividindo os dados em cada nó com base no recurso e no valor de divisão que resultem na menor impureza.

        best_gain = 0
        best_feature = None
        best_value = None
        best_sets = None

        impurity_node = impurity_func(y)

        for feature in range(num_features):
            feature_valores = np.unique(X[:, feature])
            for valor in feature_valores:
                left_indices = np.where(X[:, feature] < valor)
                right_indices = np.where(X[:, feature] >= valor)
                left_y = y[left_indices]
                right_y = y[right_indices]
                gain = self._information_gain(left_y, right_y, impurity_node)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_value = valor
                    best_sets = {
                        "left_indices": left_indices,
                        "right_indices": right_indices
                    }

        if best_gain > 0:
            left = self._build_tree(
                X[best_sets["left_indices"]], y[best_sets["left_indices"]], depth + 1)
            right = self._build_tree(
                X[best_sets["right_indices"]], y[best_sets["right_indices"]], depth + 1)
            return (best_feature, best_value, left, right)
        return np.bincount(y).argmax()

    def _information_gain(self, left_y, right_y, impurity_node):
        p = float(len(left_y)) / (len(left_y) + len(right_y))
        return impurity_node - p * self._gini_impurity(left_y) - (1 - p) * self._gini_impurity(right_y)

    def predict(self, X):

        return np.array([self._predict(inputs, self.tree) for inputs in X])

    def _predict(self, inputs, tree):
        if not isinstance(tree, tuple):
            return tree
        feature, threshold, left_subtree, right_subtree = tree
        if inputs[feature] < threshold:
            return self._predict(inputs, left_subtree)
        else:
            return self._predict(inputs, right_subtree)

    def _gini_impurity(self, y):
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

    def _entropy_impurity(self, y):
        m = len(y)
        return -sum((np.sum(y == c) / m) * np.log2(np.sum(y == c) / m) for c in np.unique(y))

# Teste

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

tree = DecisionTree(criterion='gini', max_depth=3, min_samples_split=2)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy*100}%')

Acurácia: 100.0%


## **Atividade 3 - Classificação com o dataset Breast cancer**

Features: Total de 30

Para essa atividade utilizar as features:
1. Mean radius
2. Mean texture
3. Mean perimeter
4. Mean area
5. Mean smoothness

Variável dependente: Presença de câncer de mama maligno ou benigno
1. 0:  Indica que o tumor é maligno (câncer de mama maligno).
2. 1: Indica que o tumor é benigno (câncer de mama benigno).


>**Treine os dados da base breast cancer utilizando o KNN e DT implementados**

>**Faça as predições nos dados de teste e compare os dados reais com os preditos**


In [7]:
data = datasets.load_breast_cancer()
colunas = ['Mean radius', 'Mean texture',
           'Mean perimeter', 'Mean area', 'Mean smoothness']
pd.DataFrame(data['data'][:, :5], columns=colunas)

,Mean radius,Mean texture,Mean perimeter,Mean area,Mean smoothness
0,17.99,10.38,122.80,1001.0,0.11840
1,20.57,17.77,132.90,1326.0,0.08474
2,19.69,21.25,130.00,1203.0,0.10960
3,11.42,20.38,77.58,386.1,0.14250
4,20.29,14.34,135.10,1297.0,0.10030
...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100
565,20.13,28.25,131.20,1261.0,0.09780
566,16.60,28.08,108.30,858.1,0.08455
567,20.60,29.33,140.10,1265.0,0.11780


In [8]:
from sklearn.metrics import precision_score, accuracy_score


data = datasets.load_breast_cancer()
X = data['data'][:, :5]
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# # Treinando e testando KNN
knn = KNN(k=3)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test, distance_metric='l2')

# Treinando e testando Decision Tree
tree = DecisionTree(criterion='gini', max_depth=3, min_samples_split=2)
tree.fit(X_train, y_train)
tree_predictions = tree.predict(X_test)

# Avaliando a acurácia
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions)

tree_accuracy = accuracy_score(y_test, tree_predictions)
tree_precision = precision_score(y_test, tree_predictions)

print(
    f'KNN:\n Acurácia: {knn_accuracy*100:.2f}%\n Precisão: {knn_precision*100:.2f}%')

print(
    f'\nÁrvore de decisão:\n Acurácia: {tree_accuracy*100:.2f}%\n Precisão: {tree_precision*100:.2f}%')

KNN:
 Acurácia: 90.35%
 Precisão: 90.54%

Árvore de decisão:
 Acurácia: 90.35%
 Precisão: 94.12%


>**As 5 características do dataset Breast cancer possuem escalas diferentes, o que compromete a taxa de acerto. Crie abaixo novas classes KNN_Norm e DecisionTree_Norm para aplicar um algoritmo de normalização de características, refaça o treinamento com as novas classes e analise o impacto no resultado para os dois classificadores.**

Exemplos de algoritmos de normalização que podem ser utilizados:
1. Normalização Min-Max (Min-Max normalization): Este método dimensiona os dados para que fiquem dentro de um intervalo específico, geralmente entre 0 e 1.

2. Normalização pelo Max: Divide-se cada valor pelo maior valor da amostra. Este método será válido apenas em casos em que os valores forem sempre positivos.

3. Normalização Z-Score: Este método transforma os dados para que tenham uma média zero e um desvio padrão de 1.

# **Implementando a classe KNN_Norm**

In [9]:
class KNN_Norm:

    def __init__(self, k=3, norm_method='minmax'):
        self.k = k
        self.norm_method = norm_method

    def fit(self, X_train, y_train):
        self.X_train = self.normalização(X_train)
        self.y_train = y_train

    def predict(self, X_test, distance_metric='l2'):
        X_test = self.normalização(X_test)

        if distance_metric == 'l1':
            dist_func = self._l1_distance
        elif distance_metric == 'l2':
            dist_func = self._l2_distance
        else:
            raise ValueError(
                "Métrica de distancia inválida. Escolha 'l1' ou 'l2'.")

        predictions = [self._predict(x, dist_func) for x in X_test]
        return np.array(predictions)

    def _predict(self, x, dist_func):
        distances = [dist_func(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def _l1_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))

    def _l2_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def normalização(self, X):
        if self.norm_method == 'minmax':
            return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
        elif self.norm_method == 'max':
            return X / X.max(axis=0)
        elif self.norm_method == 'zscore':
            return (X - X.mean(axis=0)) / X.std(axis=0)
        else:
            raise ValueError(
                "Método de normalização inválido. Escolha 'minmax', 'max' ou 'zscore'.")

# **Implementando a classe DecisionTree_Norm**

In [10]:
class DecisionTree_Norm:

    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2, norm_method='minmax'):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.norm_method = norm_method

    def fit(self, X, y):
        X = self._normalize(X)
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))

        if depth == self.max_depth or num_samples < self.min_samples_split or num_classes == 1:
            return np.bincount(y).argmax()

        if self.criterion == 'gini':
            impurity_func = self._gini_impurity
        elif self.criterion == 'entropy':
            impurity_func = self._entropy_impurity
        else:
            raise ValueError("Critério inválido. Escolha 'gini' ou 'entropy'.")

        best_gain = 0
        best_feature = None
        best_value = None
        best_sets = None

        impurity_node = impurity_func(y)

        for feature in range(num_features):
            feature_values = np.unique(X[:, feature])
            for value in feature_values:
                left_indices = np.where(X[:, feature] < value)[0]
                right_indices = np.where(X[:, feature] >= value)[0]
                left_y = y[left_indices]
                right_y = y[right_indices]
                gain = self._information_gain(left_y, right_y, impurity_node)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_value = value
                    best_sets = {
                        "left_indices": left_indices,
                        "right_indices": right_indices
                    }

        if best_gain > 0:
            left = self._build_tree(
                X[best_sets["left_indices"]], y[best_sets["left_indices"]], depth + 1)
            right = self._build_tree(
                X[best_sets["right_indices"]], y[best_sets["right_indices"]], depth + 1)
            return (best_feature, best_value, left, right)
        return np.bincount(y).argmax()

    def _information_gain(self, left_y, right_y, impurity_node):
        p = float(len(left_y)) / (len(left_y) + len(right_y))
        return impurity_node - p * self._gini_impurity(left_y) - (1 - p) * self._gini_impurity(right_y)

    def predict(self, X):
        X = self._normalize(X)
        return np.array([self._predict(inputs, self.tree) for inputs in X])

    def _predict(self, inputs, tree):
        if not isinstance(tree, tuple):
            return tree
        feature, threshold, left_subtree, right_subtree = tree
        if inputs[feature] < threshold:
            return self._predict(inputs, left_subtree)
        else:
            return self._predict(inputs, right_subtree)

    def _gini_impurity(self, y):
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

    def _entropy_impurity(self, y):
        m = len(y)
        return -sum((np.sum(y == c) / m) * np.log2(np.sum(y == c) / m) for c in np.unique(y))

    def _normalize(self, X):
        if self.norm_method == 'minmax':
            return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
        elif self.norm_method == 'max':
            return X / X.max(axis=0)
        elif self.norm_method == 'zscore':
            return (X - X.mean(axis=0)) / X.std(axis=0)
        else:
            raise ValueError(
                "Método de normalização inválido. Escolha 'minmax', 'max' ou 'zscore'.")

# Normalizando e comparando

In [11]:
def normalization(norm_method):
    
    knn_norm = KNN_Norm(k=3, norm_method=norm_method)
    knn_norm.fit(X_train, y_train)
    y_pred_knn_norm = knn_norm.predict(X_test)
    accuracy_knn_norm = accuracy_score(y_test, y_pred_knn_norm)
    
    tree_norm = DecisionTree_Norm(criterion='gini', max_depth=3, norm_method=norm_method)
    tree_norm.fit(X_train, y_train)
    y_pred_tree_norm = tree_norm.predict(X_test)
    accuracy_tree_norm = accuracy_score(y_test, y_pred_tree_norm)
    
    return accuracy_knn_norm, accuracy_tree_norm

methods = ['minmax', 'max', 'zscore']
results = {}
for method in methods:
    results[method] = normalization(method)

print(f"Acurácia KNN sem normalização: {knn_accuracy*100:.2f}%")
print(f"Acurácia Árvore de Decisão sem normalização: {tree_accuracy*100:.2f}%")
for method, (acc_knn, acc_tree) in results.items():
    print(f"\nAcurácia KNN com {method} normalização: {acc_knn*100:.2f}%")
    print(f"Acurácia Árvore de Decisão com {method} normalização: {acc_tree*100:.2f}%")

Acurácia KNN sem normalização: 90.35%
Acurácia Árvore de Decisão sem normalização: 90.35%

Acurácia KNN com minmax normalização: 74.56%
Acurácia Árvore de Decisão com minmax normalização: 82.46%

Acurácia KNN com max normalização: 57.02%
Acurácia Árvore de Decisão com max normalização: 74.56%

Acurácia KNN com zscore normalização: 93.86%
Acurácia Árvore de Decisão com zscore normalização: 92.11%


>**Divida o dataset em treinamento, teste e validação. Com o conjunto de validação varie os hiperparâmetros dos classificadores implementados e indique os melhores hiperparâmetros.**

Para o KNN indique por exemplo qual o melhor K vizinho (3,5,10..), qual a melhor distância (euclidiana ou manhattan). Para a DT pode variar os parâmetros max_depth, min_samples_split, grau de impureza. Como métricas de avaliação podem utilizar recall,   precision e f-measure.


In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42)


def evaluate_knn(X_train, y_train, X_val, y_val, k_values, distance_metrics):
    best_params = None
    best_score = 0

    for k in k_values:
        for metric in distance_metrics:
            knn = KNN_Norm(k=k, norm_method='minmax')
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_val, distance_metric=metric)

            recall = recall_score(y_val, y_pred, average='macro')
            precision = precision_score(y_val, y_pred, average='macro')
            f1 = f1_score(y_val, y_pred, average='macro')

            if f1 > best_score:
                best_score = f1
                best_params = {'k': k, 'metric': metric}

    return best_params, best_score


k_values = [3, 5, 10]
distance_metrics = ['l1', 'l2']
best_knn_params, best_knn_score = evaluate_knn(
    X_train, y_train, X_val, y_val, k_values, distance_metrics)

print(
    f"Melhores hiperparâmetros para KNN: {best_knn_params}, F1-Score: {best_knn_score:.2f}")


def evaluate_decision_tree(X_train, y_train, X_val, y_val, depths, min_samples, criteria):
    best_params = None
    best_score = 0

    for depth in depths:
        for min_samples_split in min_samples:
            for criterion in criteria:
                tree = DecisionTree_Norm(criterion=criterion, max_depth=depth,
                                         min_samples_split=min_samples_split, norm_method='minmax')
                tree.fit(X_train, y_train)
                y_pred = tree.predict(X_val)

                
                f1 = f1_score(y_val, y_pred, average='macro')

                if f1 > best_score:
                    best_score = f1
                    best_params = {
                        'max_depth': depth, 'min_samples_split': min_samples_split, 'criterion': criterion}

    return best_params, best_score


depths = [3, 5, 10]
min_samples = [2, 5, 10]
criteria = ['gini', 'entropy']
best_tree_params, best_tree_score = evaluate_decision_tree(
    X_train, y_train, X_val, y_val, depths, min_samples, criteria)

print(
    f"Melhores hiperparâmetros para Decision Tree: {best_tree_params}, F1-Score: {best_tree_score:.2f}")





Melhores hiperparâmetros para KNN: {'k': 10, 'metric': 'l1'}, F1-Score: 0.71
Melhores hiperparâmetros para Decision Tree: {'max_depth': 10, 'min_samples_split': 2, 'criterion': 'gini'}, F1-Score: 0.76


## **Atividade 4 - Classificação com o dataset Wine**

Features: Total de 13

Para essa atividade utilizar as features:
1. Teor alcoólico
2. Ácido málico
3. Cinzas

Variável dependente: Classe do vinho
1. Classe 0: Vinhos da primeira origem geográfica.
2. Classe 1: Vinhos da segunda origem geográfica.
3. Classe 2: Vinhos da terceira origem geográfica.


**Nesta atividade deverá ser implementada uma função que divida os dados em folds para realizar validação cruzada.**

**Também deverá ser realizado uma comparação entre um treinamento sem e com a validação cruzada no KNN e DT. Os dados deverão ser divididos em 5 folds. E como métricas de avaliação podem utilizar recall, precision e f-measure**


In [13]:
data = datasets.load_wine()
colunas = ['Teor alcoólico', 'Acidez málica', 'Cinzas']
pd.DataFrame(data['data'][:, :3], columns=colunas)

,Teor alcoólico,Acidez málica,Cinzas
0,14.23,1.71,2.43
1,13.20,1.78,2.14
2,13.16,2.36,2.67
3,14.37,1.95,2.50
4,13.24,2.59,2.87
...,...,...,...
173,13.71,5.65,2.45
174,13.40,3.91,2.48
175,13.27,4.28,2.26
176,13.17,2.59,2.37


In [14]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import datasets

def val_cruz_kfold(X, y, num_folds=5):
    kf = KFold(n_splits=num_folds)
    folds = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        folds.append((X_train, y_train, X_test, y_test))
    return folds

data = datasets.load_breast_cancer()
X = data['data'][:, :5]
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Modelos sem validação cruzada
knn = KNN(k=3)
dt = DecisionTree(criterion='gini', max_depth=3, min_samples_split=2)

knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
precision_knn = precision_score(y_test, y_pred_knn, average='macro')
recall_knn = recall_score(y_test, y_pred_knn, average='macro')
f1_knn = f1_score(y_test, y_pred_knn, average='macro')

dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
precision_dt = precision_score(y_test, y_pred_dt, average='macro')
recall_dt = recall_score(y_test, y_pred_dt, average='macro')
f1_dt = f1_score(y_test, y_pred_dt, average='macro')

# Modelos com validação cruzada
folds = val_cruz_kfold(X_train, y_train)

precision_knn_cv, recall_knn_cv, f1_knn_cv = [], [], []
precision_dt_cv, recall_dt_cv, f1_dt_cv = [], [], []

for X_train, y_train, X_test, y_test in folds:
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    precision_knn_cv.append(precision_score(y_test, y_pred_knn, average='macro'))
    recall_knn_cv.append(recall_score(y_test, y_pred_knn, average='macro'))
    f1_knn_cv.append(f1_score(y_test, y_pred_knn, average='macro'))

    dt.fit(X_train, y_train)
    y_pred_dt = dt.predict(X_test)
    precision_dt_cv.append(precision_score(y_test, y_pred_dt, average='macro'))
    recall_dt_cv.append(recall_score(y_test, y_pred_dt, average='macro'))
    f1_dt_cv.append(f1_score(y_test, y_pred_dt, average='macro'))

print(
    f'Knn sem validação cruzada: Precision = {precision_knn*100:.2f}%, Recall = {recall_knn*100:.2f}%, F1 = {f1_knn*100:.2f}%')
print(
    f'Knn com validação cruzada: Precision = {np.mean(precision_knn_cv)*100:.2f}%, Recall = {np.mean(recall_knn_cv)*100:.2f}%, F1 = {np.mean(f1_knn_cv)*100:.2f}%\n')

print(
    f'Árvore de Decisão sem validação cruzada: Precision = {precision_dt*100:.2f}%, Recall = {recall_dt*100:.2f}%, F1 = {f1_dt*100:.2f}%')
print(
    f'Árvore de Decisão com validação cruzada: Precision = {np.mean(precision_dt_cv)*100:.2f}%, Recall = {np.mean(recall_dt_cv)*100:.2f}%, F1 = {np.mean(f1_dt_cv)*100:.2f}%')


Knn sem validação cruzada: Precision = 89.47%, Recall = 87.70%, F1 = 88.45%
Knn com validação cruzada: Precision = 87.80%, Recall = 86.06%, F1 = 86.48%

Árvore de Decisão sem validação cruzada: Precision = 89.95%, Recall = 89.95%, F1 = 89.95%
Árvore de Decisão com validação cruzada: Precision = 87.50%, Recall = 85.61%, F1 = 86.14%
